In [2]:
import cv2
net = cv2.dnn.readNetFromCaffe("deploy.prototxt",
                               "res10_300x300_ssd_iter_140000.caffemodel")

from tensorflow.keras.models import load_model
model = load_model("Weights-99-0.6537.h5")

In [3]:
# pafy, youtube-dl 패키지 설치해야 함
import pafy

url = "https://youtu.be/embYkODkzcs"
video = pafy.new(url)
best = video.getbest(preftype="mp4")
print("best resolution : {}".format(best.resolution))

best resolution : 1280x720


In [6]:
import numpy as np
import pafy

EMOTIONS_LIST = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

url = "https://youtu.be/embYkODkzcs"
video = pafy.new(url)
best = video.getbest(preftype="mp4")
print("best resolution : {}".format(best.resolution))

video = cv2.VideoCapture(best.url)

while True:
    ret, frame = video.read()
    if ret:
        (h, w) = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(frame,
                                                (300, 300)), 1.0, (300, 300),
                                     (104.0, 177.0, 123.0))
        # 탐색 수행  
        net.setInput(blob)
        detections = net.forward()

        for i in range(0, detections.shape[2]): 
            # 탐지한 객체의 신뢰도를 얻어냄 
            confidence = detections[0, 0, i, 2]
            # 신뢰도50% 이상에 대해서만 박스를 그림 
            if confidence > 0.5: 

                # 객체 경계상자를 그리기 위한 좌표 계산 
                box = detections[0,0,i,3:7]*np.array([w,h,w,h])
                (startX,startY,endX,endY) = box.astype("int")
                # 경계상자를 그림
                text = "{:.2f}%".format(confidence * 100)
                y = startY-10 if startY-10 > 10 else startY+10
                cv2.rectangle(frame, (startX, startY),
                              (endX, endY), (0,0,255), 2)
                face = frame[startY:endY, startX:endX]
                roi = cv2.resize(face, (48, 48))
                roi_gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
                
                pred = model.predict(roi_gray[np.newaxis, :, :, np.newaxis])
                emotion = EMOTIONS_LIST[np.argmax(pred)]
                cv2.putText(frame, emotion, (startX, y), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1,
                            (0,0,255), 3)

        cv2.imshow("Emotion", frame)

        if cv2.waitKey(1)&0xFF == 27: # ESC
            video.release()
            cv2.destroyAllWindows()

    else:
        video.release()
        break